In [55]:
import json
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [56]:
LOG_DATA_PATH = "data\\added_action_id.json"

# 중첩 구조 평탄화해서 읽기
with open(LOG_DATA_PATH, "r", encoding="utf-8") as f:
    # data = json.load(f)
    data = [json.loads(line.strip()) for line in f if line.strip()]  # .jsonl일 때 읽어오는 방법
    
df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   event                 79 non-null     object
 1   userId                79 non-null     object
 2   username              79 non-null     object
 3   timestamp             79 non-null     object
 4   workspaceId           79 non-null     int64 
 5   details.endDate       51 non-null     object
 6   details.importance    79 non-null     int64 
 7   details.name          79 non-null     object
 8   details.state         79 non-null     object
 9   details.startDate     50 non-null     object
 10  details.participants  79 non-null     object
 11  details.actionId      79 non-null     int64 
dtypes: int64(3), object(9)
memory usage: 7.5+ KB


In [57]:
# actionId 단위의 최신 이벤트만
latest_actions = (
    df
    .sort_values("timestamp", ascending=False)
    .drop_duplicates(
        subset=["workspaceId", "details.actionId"],
        keep="first"
    )
)

# stat 1/2 공통 작업 - participants 펼치기
exploded = latest_actions.explode('details.participants')
exploded['participants_userId'] = exploded['details.participants'].apply(lambda x: x.get('userId') if isinstance(x, dict) else None)

# 필요한 필드 정리
filtered = exploded[[
    'event',
    'userId',
    'participants_userId',
    'timestamp',
    'workspaceId',
    'details.actionId',
    'details.name',
    'details.state',
    'details.importance',
    'details.startDate',
    'details.endDate',
]]

filtered = filtered.dropna(subset=['participants_userId'])

# action DELETE 이벤트가 발생한 actions Id
deleted_ids = latest_actions[latest_actions["event"] == "DELETE_PROJECT_PROGRESS_ACTION"]["details.actionId"].unique()

# DELETE 이벤트 발생했던 actionId 제거
filtered = filtered[~filtered["details.actionId"].isin(deleted_ids)]

# Dtype 정리
filtered['userId'] = filtered['userId'].astype(int)
filtered['participants_userId'] = filtered['participants_userId'].astype(int)
filtered['timestamp'] = pd.to_datetime(filtered['timestamp'])
filtered['details.startDate'] = pd.to_datetime(filtered['details.startDate'])
filtered['details.endDate'] = pd.to_datetime(filtered['details.endDate'])

# 모든 사용자 ID (이벤트 발생자 + 참여자)
all_user_ids = set(filtered['userId'].unique()) | set(filtered['participants_userId'].unique())

user_dfs = {}
for uid in all_user_ids:
    # 해당 사용자가 발생시킨 이벤트 OR 참여한 이벤트
    user_data = filtered[(filtered['userId'] == uid) | (filtered['participants_userId'] == uid)].copy()
    user_dfs[f'df_{uid}'] = user_data

In [58]:
user_dfs

{'df_1':                              event  userId  participants_userId  \
 15    DONE_PROJECT_PROGRESS_ACTION      48                    1   
 12  CREATE_PROJECT_PROGRESS_ACTION       1                    1   
 
                              timestamp  workspaceId  details.actionId  \
 15 2025-06-26 11:27:55.035862722+00:00            1                 3   
 12 2025-06-25 14:36:54.288403129+00:00            1                 2   
 
                    details.name details.state  details.importance  \
 15  Postgresql에 통계 데이터 저장 구조 설계          DONE                   3   
 12            회원 프론트 화면 피그마 디자인   IN_PROGRESS                   3   
 
    details.startDate details.endDate  
 15               NaT             NaT  
 12               NaT             NaT  ,
 'df_5':                            event  userId  participants_userId  \
 34  DONE_PROJECT_PROGRESS_ACTION       5                    5   
 33  UPDATE_PARTICIPANT_TO_ACTION       5                    5   
 33  UPDATE_PARTICIPANT

In [ ]:
# ===== Statistics 1 Start =====

# state, importance 기준 grouping -> count 목적
# 이벤트 발생자와 참여자 모두 집계
all_stat1_results = []

for user_df in user_dfs.values():
    # 발생자 데이터 준비
    initiator_data = user_df[['userId', 'details.state', 'details.importance', 'details.actionId']].copy()
    initiator_data['role'] = 'initiator'
    initiator_data = initiator_data.rename(columns={'userId': 'final_userId'})
    
    # 참여자 데이터 준비  
    participant_data = user_df[['participants_userId', 'details.state', 'details.importance', 'details.actionId']].copy()
    participant_data['role'] = 'participant'
    participant_data = participant_data.rename(columns={'participants_userId': 'final_userId'})
    
    # 합치기
    user_all_data = pd.concat([initiator_data, participant_data], ignore_index=True)
    all_stat1_results.append(user_all_data)

# 모든 유저 데이터 합치기
if all_stat1_results:
    combined_stats = pd.concat(all_stat1_results, ignore_index=True)
    
    # 중복 제거: 같은 final_userId, state, importance, actionId 조합의 중복 행 제거
    dedup_stats = combined_stats.drop_duplicates(
        subset=['final_userId', 'details.state', 'details.importance', 'details.actionId']
    )
    
    # 최종 집계
    stat1_result = (
        dedup_stats
        .groupby(['final_userId', 'details.state', 'details.importance'])
        .size()
        .reset_index(name='count')
        .rename(columns={'final_userId': 'userId'})
        .to_dict(orient='records')
    )
else:
    stat1_result = []

stat1_result

[{'userId': 1, 'details.state': 'DONE', 'details.importance': 3, 'count': 1},
 {'userId': 1,
  'details.state': 'IN_PROGRESS',
  'details.importance': 3,
  'count': 1},
 {'userId': 5, 'details.state': 'DONE', 'details.importance': 1, 'count': 2},
 {'userId': 5, 'details.state': 'DONE', 'details.importance': 2, 'count': 3},
 {'userId': 5, 'details.state': 'DONE', 'details.importance': 5, 'count': 2},
 {'userId': 6, 'details.state': 'DONE', 'details.importance': 1, 'count': 2},
 {'userId': 6, 'details.state': 'DONE', 'details.importance': 2, 'count': 2},
 {'userId': 6, 'details.state': 'DONE', 'details.importance': 4, 'count': 1},
 {'userId': 13, 'details.state': 'DONE', 'details.importance': 1, 'count': 1},
 {'userId': 13, 'details.state': 'DONE', 'details.importance': 2, 'count': 2},
 {'userId': 13, 'details.state': 'DONE', 'details.importance': 4, 'count': 1},
 {'userId': 48, 'details.state': 'DONE', 'details.importance': 3, 'count': 1}]

In [60]:
# ===== Statistics 2 Start=====
# DONE 상태의 action만 뽑아오기
done_df = filtered[filtered['details.state'] == 'DONE'].copy()

# 날짜형 변환 및 duration 계산
done_df['details.startDate'] = pd.to_datetime(done_df['details.startDate'], utc=True)
done_df['details.endDate'] = pd.to_datetime(done_df['details.endDate'], utc=True)
done_df['duration_hours'] = ((done_df['timestamp'] - done_df['details.startDate']).dt.total_seconds() / 3600)

# 결측치, 음수 제거
done_df = done_df.dropna(subset=['duration_hours'])
done_df = done_df[done_df['duration_hours'] >= 0]

# 참여자용 중복 제거
participants_done_df = (
    done_df
    .sort_values('timestamp', ascending=False)
    .drop_duplicates(
        subset=['workspaceId', 'details.actionId', 'participants_userId'],
        keep="first"
    )
)

# 발생자용 중복 제거  
initiator_done_df = (
    done_df
    .sort_values('timestamp', ascending=False)
    .drop_duplicates(
        subset=['workspaceId', 'details.actionId', 'userId'],
        keep="first"
    )
)

# 참여자 통계
participants_result = (
    participants_done_df
    .groupby(['participants_userId', 'details.importance'])
    ['duration_hours']
    .mean()
    .reset_index(name='mean_hours')
    .rename(columns={'participants_userId': 'userId'})
)
participants_result['role'] = 'participant'

# 발생자 통계
initiator_result = (
    initiator_done_df
    .groupby(['userId', 'details.importance'])
    ['duration_hours']
    .mean()
    .reset_index(name='mean_hours')
)
initiator_result['role'] = 'initiator'

# 결과 통합 (role 정보 유지)
final_result = pd.concat([participants_result, initiator_result], ignore_index=True)
final_result = final_result.drop('role', axis=1)
stat2_result = final_result.to_dict(orient='records')

stat2_result

[{'userId': 5, 'details.importance': 1, 'mean_hours': 26.732496422246115},
 {'userId': 5, 'details.importance': 2, 'mean_hours': 86.85750967858945},
 {'userId': 6, 'details.importance': 1, 'mean_hours': 26.732496422246115},
 {'userId': 6, 'details.importance': 2, 'mean_hours': 98.71575865647667},
 {'userId': 6, 'details.importance': 4, 'mean_hours': 557.9989783761825},
 {'userId': 13, 'details.importance': 1, 'mean_hours': 26.732496422246115},
 {'userId': 13, 'details.importance': 2, 'mean_hours': 86.85750967858945},
 {'userId': 13, 'details.importance': 4, 'mean_hours': 557.9989783761825},
 {'userId': 5, 'details.importance': 2, 'mean_hours': 74.99926070070222},
 {'userId': 6, 'details.importance': 1, 'mean_hours': 26.732496422246115},
 {'userId': 6, 'details.importance': 4, 'mean_hours': 557.9989783761825},
 {'userId': 13, 'details.importance': 2, 'mean_hours': 98.71575865647667}]